In [1]:
import torch 
import torch.nn as nn
import math
import numpy as np

In [2]:
class vitri_Encoding(nn.Module):
    def __init__(self, kichthuoc_vector,dodaitoidachuoi):
        super(vitri_Encoding, self).__init__()
        vt_en=torch.zeros(dodaitoidachuoi, kichthuoc_vector)
        vt=torch.arange(0, dodaitoidachuoi, dtype=torch.float).unsqueeze(1)
        hschia=torch.exp(torch.arange(0, kichthuoc_vector, 2).float() * -(math.log(10000.0) / kichthuoc_vector))
        vt_en[:, 0::2] = torch.sin(vt * hschia)
        vt_en[:, 1::2] = torch.cos(vt * hschia)
        vt_en = vt_en.unsqueeze(0)
        self.register_buffer('vt_en', vt_en)
    def forward(self, x):
        return x + self.vt_en[:, :x.size(1), :]

In [3]:
class khoi_tranformer_Encoder(nn.Module):
    def __init__ (self, kichthuoc_vector,soluong_dauvao,kichthuoc_tang,tile_dropout=0.1):
        super(khoi_tranformer_Encoder, self).__init__()
        self.tang_dauvao_chu_y=nn.MultiheadAttention(
            embed_dim=kichthuoc_vector,
            num_heads=soluong_dauvao,
            dropout=tile_dropout
        )
        self.mang_motchieu=nn.Sequential(
            nn.Linear(kichthuoc_vector, kichthuoc_tang),
            nn.ReLU(),
            nn.Linear(kichthuoc_tang, kichthuoc_vector)
        )
        self.tangchuanhoa1=nn.LayerNorm(kichthuoc_vector, eps=1e-6)
        self.tangchuanhoa2=nn.LayerNorm(kichthuoc_vector, eps=1e-6)
        self.tang_dropout1=nn.Dropout(tile_dropout)
        self.tang_dropout2=nn.Dropout(tile_dropout)
    def forward(self, x,mask=None):
        chu_y, _ = self.tang_dauvao_chu_y(x, x, x, attn_mask=mask)
        chu_y=self.tang_dropout1(chu_y)
        dulieura1= x + chu_y
        dulieura1=self.tangchuanhoa1(dulieura1)
        dulieura2=self.mang_motchieu(dulieura1)
        dulieura2=self.tang_dropout2(dulieura2)
        dulieura2 = dulieura1 + dulieura2
        dulieura2=self.tangchuanhoa2(dulieura2)
        return dulieura2

In [4]:
class lop_tranformer_Encoder(nn.Module):
    def __init__(self, soluong_lop, kichthuoc_vector, soluong_dauvao,
                kichthuoc_tang, soluong_tuvung,
                dodaichuoi_toida,tile_dropout=0.1):
        
        super(lop_tranformer_Encoder, self).__init__()
        self.kichthuoc_vector = kichthuoc_vector
        self.tang_nhung=nn.Embedding(soluong_tuvung, kichthuoc_vector)
        self.mahoa_vitri = vitri_Encoding(kichthuoc_vector, dodaichuoi_toida)
        self.danhsach_khoi_xuly= nn.ModuleList([
            khoi_tranformer_Encoder(
                kichthuoc_vector, soluong_dauvao, 
                kichthuoc_tang, tile_dropout) for _ in range(soluong_lop)
            ])
        self.lop_dropout = nn.Dropout(tile_dropout)
    def forward(self, x, mask=None):
        x= self.tang_nhung(x)* math.sqrt(self.kichthuoc_vector)
        x = self.mahoa_vitri(x)
        x = self.lop_dropout(x)
        for khoi_xuly in self.danhsach_khoi_xuly:
            x = khoi_xuly(x, mask)
        return x

In [5]:
def tao_mohinh_tranformer_Encoder(soluong_lop, kichthuoc_vector, soluong_dauvao,
                            kichthuoc_tang, soluong_tuvung,
                            dodaichuoi_toida, tile_dropout=0.1):
    return lop_tranformer_Encoder(
        soluong_lop, kichthuoc_vector, soluong_dauvao,
        kichthuoc_tang, soluong_tuvung,
        dodaichuoi_toida, tile_dropout
    )

In [6]:
#Tham số mô hình
soluong_lop = 2
kichthuoc_vector = 64
soluong_dauvao = 4
kichthuoc_tang = 128
soluong_tuvung = 1000
dodaichuoi_toida = 100 
tile_dropout = 0.1
# Tạo mô hình
mohinh = tao_mohinh_tranformer_Encoder(
    soluong_lop, kichthuoc_vector, soluong_dauvao,
    kichthuoc_tang, soluong_tuvung,
    dodaichuoi_toida, tile_dropout
)
mohinh.eval()  # Chuyển mô hình sang chế độ đánh giá

lop_tranformer_Encoder(
  (tang_nhung): Embedding(1000, 64)
  (mahoa_vitri): vitri_Encoding()
  (danhsach_khoi_xuly): ModuleList(
    (0-1): 2 x khoi_tranformer_Encoder(
      (tang_dauvao_chu_y): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
      )
      (mang_motchieu): Sequential(
        (0): Linear(in_features=64, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=64, bias=True)
      )
      (tangchuanhoa1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
      (tangchuanhoa2): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
      (tang_dropout1): Dropout(p=0.1, inplace=False)
      (tang_dropout2): Dropout(p=0.1, inplace=False)
    )
  )
  (lop_dropout): Dropout(p=0.1, inplace=False)
)

In [7]:
sample_data = torch.tensor([[0, 1, 4, 3, 2, 3, 2, 3, 5, 4, 5, 6]], dtype=torch.long)

padding_mask = (sample_data == 0).float() * -1e9
padding_mask = padding_mask.unsqueeze(1).unsqueeze(2)

with torch.no_grad():
    output = mohinh(sample_data, mask=None)
    print("Kích thước đầu ra:", output.shape)
    print("Đầu ra mẫu:", output[0, 0, :5])

Kích thước đầu ra: torch.Size([1, 12, 64])
Đầu ra mẫu: tensor([-0.7253,  0.2042, -1.4659,  1.6815, -0.0865])
